# Imports

In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import string
from matplotlib.patches import Rectangle
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from ipywidgets import interact, widgets
import os
from scipy.interpolate import interp1d, griddata
from scipy.signal import savgol_filter
from functions.Hilfsfunktionen_plot import plot_component_with_sem

# Config

In [11]:
config = {}

# ROMY coordinates
config['ROMY_lon'] = 11.275501
config['ROMY_lat'] = 48.162941

# path for figures to store
config['outpath_figs'] = "C:/Bachelorarbeit/figures/Geschwindigkeit/ADRRLAS/"
# path for output data
config['outpath_data'] = "C:/Bachelorarbeit/data/waveformsadrrlas/"
outpath = config['outpath_figs']
font_title=17
font_ax=13
font_leg=11

# Hilfsfunktionen


In [18]:
def plot_component(ax, df, title):
    ratio_cols = [col for col in df.columns if col.startswith("ratio_")]
    frequencies = [float(col.split("_")[1][:-2]) for col in ratio_cols]

    # Plot nach Tiefenbereichen
    for (dmin, dmax), label in zip(depth_bins, depth_labels):
        df_subset = df[(df["depth"] >= dmin) & (df["depth"] < dmax)]
        num_events = len(df_subset)
        if df_subset.empty:
            continue

        mean_ratios = df_subset[ratio_cols].mean()
        sem_ratios = df_subset[ratio_cols].sem()
        legend_label = f"depth: {label} (n = {num_events})"

        if dmin == 0 and dmax == 700:
            ax.plot(frequencies, mean_ratios.values, linestyle='--', linewidth=2, label=legend_label)
        else:
            ax.plot(frequencies, mean_ratios.values, linestyle='-', label=legend_label)

        ax.fill_between(frequencies,
                        mean_ratios.values - sem_ratios.values,
                        mean_ratios.values + sem_ratios.values,
                        alpha=0.2)

    # Gesamtmittelwert über alle Tiefen (ohne SEM)
    overall_mean = df[ratio_cols].mean()
    ax.plot(frequencies,
            overall_mean.values,
            color='black',
            linestyle=':',
            linewidth=2,
            label='Overall mean (all depths)')

    ax.set_xlabel("Frequency [Hz]",fontsize=font_ax)
    ax.set_ylabel("Median ratio (ROMY (ADR)/RLAS)",fontsize=font_ax)
    ax.set_title(title,fontsize=font_title)
    ax.grid(True)
    ax.legend(loc='best')

In [19]:
# Hilfsfunktion zum Zeichnen eines Rechtecks
def add_region(ax, lon_min, lon_max, lat_min, lat_max, label, color='red', alpha=0.2):
    rect = Rectangle((lon_min, lat_min), lon_max - lon_min, lat_max - lat_min,
                     linewidth=1, edgecolor=color, facecolor=color, alpha=alpha,
                     transform=ccrs.PlateCarree(), label=label)
    ax.add_patch(rect)

# Plots

In [20]:
fig, ax = plt.subplots(figsize=(10, 6))
df_T = pd.read_csv(config['outpath_figs'] + "spektralanalyse_ergebnisse_oadr.csv")
df_Z = pd.read_csv(config['outpath_figs'] + "spektralanalyse_ergebnisse_aadr.csv")
# Komponenten plotten
plot_component_with_sem(ax, df_T, color='blue', label='OADR')
plot_component_with_sem(ax, df_Z, color='green', label='AADR')

# Achsen und Titel
ax.set_xlabel("Frequency [Hz]",fontsize=font_ax)
ax.set_ylabel("Mean ratio (ROMYADR/RLAS)",fontsize=font_ax)
ax.set_title("Mean ratio per frequency (± SEM)",fontsize=font_title)
ax.grid(True)
ax.legend(loc='best', fontsize=font_leg)

# Layout & speichern
plt.tight_layout()
plt.savefig(outpath + "meanratio_freqency_TandZ_SEM_adrrlas.png", dpi=300)
plt.close()


## Tiefen


In [21]:
# Konfiguration
depth_bins = [(0, 100), (100, 700)]
depth_labels = ["0–100 km", "100–700 km"]
full_range = (0, 700)


In [22]:
# Daten laden
df_oadr = pd.read_csv(outpath + "spektralanalyse_ergebnisse_oadr.csv")
df_aadr = pd.read_csv(outpath + "spektralanalyse_ergebnisse_aadr.csv")

# Subplots erstellen
fig, axs = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

# Komponenten plotten
plot_component(axs[0], df_oadr, "OADR")
plot_component(axs[1], df_aadr, "AADR")
axs[0].text(0.02, 0.95, "a)", transform=axs[0].transAxes,
            fontsize=14, fontweight='bold', va='top', ha='left')
axs[1].text(0.02, 0.95, "b)", transform=axs[1].transAxes,
            fontsize=14, fontweight='bold', va='top', ha='left')
# Formatierung & Speichern
plt.suptitle("Median ratio (ROMYADR/RLAS) per frequency divided by depth range (± SEM)")
plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.savefig(outpath + "medianratio_freqency_depths_adr.png", dpi=300)

plt.close()

## Weltkarte

In [17]:
df = pd.read_csv(outpath + "spektralanalyse_ergebnisse_oadr.csv")
# Daten aus den Spalten extrahieren
lats = df['lat'].values
lons = df['lon'].values
magnitudes = df['magnitude'].values
num=len(df)
mapplot = plt.figure(figsize=(14, 8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_global()

ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.LAND, facecolor='lightgray')

# Station-Koordinaten
sta_lat = config['ROMY_lat']
sta_lon = config['ROMY_lon']
ax.scatter(sta_lon, sta_lat, color='blue', s=100, marker='*', edgecolor='k', transform=ccrs.PlateCarree(), label='FUR')

ax.legend(loc='upper right',fontsize=font_leg)

sc = ax.scatter(lons, lats, c=magnitudes, cmap='viridis', s=[m**3 for m in magnitudes], alpha=0.7, edgecolor='k', transform=ccrs.PlateCarree())

ax.set_title(f"Earthquake distribution ({num} events)",fontsize=font_title)
ax.gridlines(draw_labels=True)

cbar = plt.colorbar(sc, ax=ax, orientation='vertical', shrink=0.8, pad=0.05)
cbar.set_label("Magnitude",fontsize=font_ax)

# Regionen markieren
add_region(ax, -50, 50, 30, 60, "Region 1 (Europe)", color="red")
add_region(ax, 60, 180, 15, 60, "Region 2 (Asia)", color="blue")
add_region(ax, 100, 180, -10, 15, "Region 2 (Asia)", color="blue")
add_region(ax, -150, -60, 5, 60, "Region 4 (Northamerica)", color="green")
add_region(ax, -120, -50, -80, 5, "Region 4 (Southamerica)", color="green")
add_region(ax, 90, 180, -60, -10, "Region 6 (Ozeania)", color="orange")
add_region(ax, -180, -150, -60, -10, "Region 6 (Ozeania)", color="orange")

# Duplikate in Legende vermeiden
handles, labels = ax.get_legend_handles_labels()
by_label = dict(zip(labels, handles))
ax.legend(by_label.values(), by_label.keys(), loc='lower left',fontsize=font_leg)

mapplotpath = config['outpath_figs'] + "worldmap_earthquke_distribution_adr.png"
mapplot.savefig(mapplotpath, dpi=150, bbox_inches='tight')
print(f" -> Kartenplot gespeichert unter: {mapplotpath}")
plt.close()

 -> Kartenplot gespeichert unter: C:/Bachelorarbeit/figures/Geschwindigkeit/ADRRLAS/worldmap_earthquke_distribution_adr.png
